In [4]:
import notebook_path
import pandas as pd
from settings import ANNOTATIONS
from pathlib import Path
from util import Dataset
import os 
import librosa

C:\working\acoustic_AL\acoustic-AL


In [15]:
df = pd.read_csv(Path(ANNOTATIONS) / 'initial_dataset_7depl.csv')
ds = Dataset()
summ_df = ds.get_deployment_summary(1, 1)
summ_df

,DATE,TIME,LAT,Unnamed: 3,LON,Unnamed: 5,POWER(V),TEMP(C),#FSFILES,#ZCFILES,#SCRUBBED
0,2023-Mar-13,14:09:00,20.43884,S,57.37490,E,5.9,28.5,0,0,0
1,2023-Mar-16,00:00:00,20.43709,S,57.37168,E,5.5,22.0,0,0,0
2,2023-Mar-16,00:01:00,20.43709,S,57.37168,E,5.5,22.0,0,0,0
3,2023-Mar-16,00:02:00,20.43709,S,57.37168,E,5.5,22.0,0,0,0
4,2023-Mar-16,00:03:00,20.43709,S,57.37168,E,5.5,22.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
2867,2023-Apr-01,06:36:00,20.43709,S,57.37168,E,5.1,22.5,0,0,0
2868,2023-Apr-01,06:37:00,20.43709,S,57.37168,E,5.1,22.5,0,0,0
2869,2023-Apr-01,06:38:00,20.43709,S,57.37168,E,5.1,22.5,0,0,0
2870,2023-Apr-01,06:39:00,20.43709,S,57.37168,E,5.1,22.5,0,0,0


In [18]:
from datetime import datetime 

datetime.strptime('31/01/22 23:59:59.999999',
                  '%d/%m/%y %H:%M:%S.%f')

def appl(row):
    return datetime.strptime(row["DATE"] + "_" + row["TIME"],
                      "%Y-%b-%d_%H:%M:%S")

summ_df["datetime"] = summ_df.apply(appl, axis=1)
summ_df 

,DATE,TIME,LAT,Unnamed: 3,LON,Unnamed: 5,POWER(V),TEMP(C),#FSFILES,#ZCFILES,#SCRUBBED,datetime
0,2023-Mar-13,14:09:00,20.43884,S,57.37490,E,5.9,28.5,0,0,0,2023-03-13 14:09:00
1,2023-Mar-16,00:00:00,20.43709,S,57.37168,E,5.5,22.0,0,0,0,2023-03-16 00:00:00
2,2023-Mar-16,00:01:00,20.43709,S,57.37168,E,5.5,22.0,0,0,0,2023-03-16 00:01:00
3,2023-Mar-16,00:02:00,20.43709,S,57.37168,E,5.5,22.0,0,0,0,2023-03-16 00:02:00
4,2023-Mar-16,00:03:00,20.43709,S,57.37168,E,5.5,22.0,0,0,0,2023-03-16 00:03:00
...,...,...,...,...,...,...,...,...,...,...,...,...
2867,2023-Apr-01,06:36:00,20.43709,S,57.37168,E,5.1,22.5,0,0,0,2023-04-01 06:36:00
2868,2023-Apr-01,06:37:00,20.43709,S,57.37168,E,5.1,22.5,0,0,0,2023-04-01 06:37:00
2869,2023-Apr-01,06:38:00,20.43709,S,57.37168,E,5.1,22.5,0,0,0,2023-04-01 06:38:00
2870,2023-Apr-01,06:39:00,20.43709,S,57.37168,E,5.1,22.5,0,0,0,2023-04-01 06:39:00


In [8]:
recordings_df = pd.DataFrame()
recordings_df["recording"] = ds.get_recordings(1, 1)
recordings_df["duration"] = recordings_df["recording"].apply(
    lambda x: ds.get_duration(x, 1, 1, sr=1_000)
)
print(recordings_df)

                 recording  duration
0    1_20230316_000000.wav     600.0
1    1_20230316_050000.wav     600.0
2    1_20230316_053000.wav     600.0
3    1_20230316_060000.wav     600.0
4    1_20230316_063000.wav     600.0
..                     ...       ...
256  1_20230401_000000.wav     600.0
257  1_20230401_050000.wav     600.0
258  1_20230401_053000.wav     600.0
259  1_20230401_060000.wav     600.0
260  1_20230401_063032.wav     568.0

[261 rows x 2 columns]


In [21]:
def extract_datetime(filename):
    filename = filename.rstrip(".wav")
    s  = filename.split('_')
    timestamp_str = s[1] + s[2]  # Extract '20230316_000000'
    dt_obj = datetime.strptime(timestamp_str, '%Y%m%d%H%M%S')
    return dt_obj

# Add a new column 'datetime'
recordings_df['datetime'] = recordings_df['recording'].apply(lambda x: extract_datetime(x))
recordings_df

,recording,duration,datetime
0,1_20230316_000000.wav,600.0,2023-03-16 00:00:00
1,1_20230316_050000.wav,600.0,2023-03-16 05:00:00
2,1_20230316_053000.wav,600.0,2023-03-16 05:30:00
3,1_20230316_060000.wav,600.0,2023-03-16 06:00:00
4,1_20230316_063000.wav,600.0,2023-03-16 06:30:00
...,...,...,...
256,1_20230401_000000.wav,600.0,2023-04-01 00:00:00
257,1_20230401_050000.wav,600.0,2023-04-01 05:00:00
258,1_20230401_053000.wav,600.0,2023-04-01 05:30:00
259,1_20230401_060000.wav,600.0,2023-04-01 06:00:00


In [23]:
summ_df.head(30)

,DATE,TIME,LAT,Unnamed: 3,LON,Unnamed: 5,POWER(V),TEMP(C),#FSFILES,#ZCFILES,#SCRUBBED,datetime
0,2023-Mar-13,14:09:00,20.43884,S,57.37490,E,5.9,28.5,0,0,0,2023-03-13 14:09:00
1,2023-Mar-16,00:00:00,20.43709,S,57.37168,E,5.5,22.0,0,0,0,2023-03-16 00:00:00
2,2023-Mar-16,00:01:00,20.43709,S,57.37168,E,5.5,22.0,0,0,0,2023-03-16 00:01:00
3,2023-Mar-16,00:02:00,20.43709,S,57.37168,E,5.5,22.0,0,0,0,2023-03-16 00:02:00
4,2023-Mar-16,00:03:00,20.43709,S,57.37168,E,5.5,22.0,0,0,0,2023-03-16 00:03:00
5,2023-Mar-16,00:04:00,20.43709,S,57.37168,E,5.5,22.0,0,0,0,2023-03-16 00:04:00
6,2023-Mar-16,00:05:00,20.43709,S,57.37168,E,5.5,22.0,0,0,0,2023-03-16 00:05:00
7,2023-Mar-16,00:06:00,20.43709,S,57.37168,E,5.5,22.0,0,0,0,2023-03-16 00:06:00
8,2023-Mar-16,00:07:00,20.43709,S,57.37168,E,5.5,22.0,0,0,0,2023-03-16 00:07:00
9,2023-Mar-16,00:08:00,20.43709,S,57.37168,E,5.5,22.0,0,0,0,2023-03-16 00:08:00


In [33]:
from settings import ANNOTATIONS

annotations = pd.read_csv(Path(ANNOTATIONS) / 'initial_dataset_7depl.csv')

def extract_datetime(filename):
    filename = filename.rstrip(".wav")
    s  = filename.split('_')
    timestamp_str = s[1] + s[2]  # Extract '20230316_000000'
    dt_obj = datetime.strptime(timestamp_str, '%Y%m%d%H%M%S')
    return dt_obj

annotations["datetime"] = annotations["recording"].apply(lambda x: extract_datetime(x))
annotations = annotations.drop(columns=['Unnamed: 0'])
annotations

,peak_time,xcorrcoef,min_t,max_t,min_f,max_f,recording,deployment,site,label,datetime
0,364.624000,0.584717,364.160667,365.087333,5531.00000,10125.000000,1_20230317_070000.wav,1.0,1.0,fast_trill_6khz,2023-03-17 07:00:00
1,215.178667,0.658409,214.715333,215.642000,5531.00000,10125.000000,1_20230319_063000.wav,1.0,1.0,fast_trill_6khz,2023-03-19 06:30:00
2,526.810667,0.604653,526.347333,527.274000,5531.00000,10125.000000,1_20230320_063000.wav,1.0,1.0,fast_trill_6khz,2023-03-20 06:30:00
3,544.922667,0.663969,544.459333,545.386000,5531.00000,10125.000000,1_20230320_063000.wav,1.0,1.0,fast_trill_6khz,2023-03-20 06:30:00
4,449.312000,0.610269,448.848667,449.775333,5531.00000,10125.000000,1_20230320_073000.wav,1.0,1.0,fast_trill_6khz,2023-03-20 07:30:00
...,...,...,...,...,...,...,...,...,...,...,...
2185,384.869333,0.654130,384.514667,385.224000,205.69162,617.953796,1_20230908_070000.wav,7.0,1.0,upsweep_500hz,2023-09-08 07:00:00
2186,386.693333,0.650236,386.338667,387.048000,205.69162,617.953796,1_20230908_070000.wav,7.0,1.0,upsweep_500hz,2023-09-08 07:00:00
2187,445.056000,0.651619,444.701333,445.410667,205.69162,617.953796,1_20230908_070000.wav,7.0,1.0,upsweep_500hz,2023-09-08 07:00:00
2188,446.906667,0.683734,446.552000,447.261333,205.69162,617.953796,1_20230908_070000.wav,7.0,1.0,upsweep_500hz,2023-09-08 07:00:00


In [35]:
annotations.to_csv(Path(ANNOTATIONS) / 'initial_dataset_7depl.csv')